In [1]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [2]:
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [3]:
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"

@everywhere atis_data = open("atis_data2.jsz","r") do fh
    deserialize(fh)    
end


In [4]:
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere  @pyimport nltk.probability as nltk_prob


@everywhere kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, atis_data["trigrams"]))
@everywhere function trigram_model(given1::AbstractString, given2::AbstractString, event::AbstractString)
    kn_prob_dist[:prob]((given1, given2, event))
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: Us

In [5]:
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}

@everywhere function transition_prob(cur_state::State{S}, next_word::S)
    trigram_model(cur_state..., next_word)
end


@everywhere _get_options_cache_type = Dict{Tuple{State{S}, Any}, Vector{Tuple{plist{S}, Float64}}}
@everywhere _get_options_caches = Dict{Vector{S}, _get_options_cache_type}()

#"""
#returns all possible orderings of the remaining words.
#the freewords are also inserted into every possible position, EXCEPT at the end.
#Freewords also will not be inserted after other freewords
#(For now)
#"""
@everywhere function get_all_orders(free_words::Vector{S}, unordered_words::Vector{S}; ϵ=0.0)
    _get_options_cache = get!(_get_options_cache_type, _get_options_caches, free_words)
    #different cache if different freewords
    
    function get_options(cur_state::State{S})
        #Can never End in a freeword
        tp = transition_prob(cur_state, END_MARKER1)
        # Given P(END_MARKER2 | curstatep[2]==END_MARKER1) = 1.0
        # Do not need to consider P(END_MARKER2 | curstatep[2]==END_MARKER1, curstatep[1])
        [
            (EmptyList{S}(), tp)
        ]
    end
    
    function get_options(state::State{S}, remaining_words::S...)
        get!(_get_options_cache, (state, remaining_words)) do
            _get_options(state, remaining_words...)
        end
    end

    function _get_options_inner(cur_state, word, remaining_words, get_option_fun)
        tp = transition_prob(cur_state, word) 
        next_state = (cur_state[2], word)
        
        if tp<=ϵ #Stop early if probability is too low
            return
        end

        for (tail, next_tp) in get_option_fun(next_state, remaining_words...)
            produce(cons(word, tail),tp*next_tp)
        end
    end

    function _get_options_nofree(cur_state::State{S}, remaining_words::S...)
        (@task begin        
            for ii in 1:length(remaining_words)
                word = remaining_words[ii]
                new_remaining_words = remaining_words[[1:ii-1; ii+1:end]]
                
                _get_options_inner(cur_state, word, new_remaining_words, get_options)
            end
        end)
    end
    
    function _get_options_free(cur_state::State{S}, remaining_words::S...)
        @task begin        
            for word in free_words
                #Important: get_options_free, only chains to _get_options_nofree
                _get_options_inner(cur_state, word, remaining_words, _get_options_nofree)
            end
        end
    end
    
    function _get_options(cur_state::State{S}, remaining_words::S...)
        chain(_get_options_nofree(cur_state, remaining_words...),
             _get_options_free(cur_state, remaining_words...)) |> collect
    end
   
    initial_state = (START_MARKER1, START_MARKER2)
    get_options(initial_state, unordered_words...)
end

In [30]:
string([1,2])

"[1,2]"

In [31]:
@everywhere function order(unordered_words::Vector{S}, free_words=S[]::Vector{S}; ϵ=0.0::Float64, best_n=1::Int)
    orders_and_probs = get_all_orders(free_words, unordered_words, ϵ=0.0)
    best_n = min(best_n, length(orders_and_probs))
    if best_n==0 #None found
        warn("No possible Orderding found. Defaulting to unordered: ".*string(unordered_words))
        unordered_words, 0.0
    elseif best_n==1 #4x as fast as the else
        max_prob, max_ii = @pipe orders_and_probs |> map(op->op[2],_) |> findmax
        orders_and_probs[max_ii], max_prob
    else
        select!(orders_and_probs, 1:best_n, by=op->-op[2] )
    end
end

@everywhere function order(unordered_words::Vector{S},free_words=S[]::Vector{S}, ϵ=0.0,best_n=1::Int)
    order(unordered_words,free_words; ϵ=0.0, best_n=best_n)[1][1]
end

In [43]:
unordered_output, test_set = open("atis_res_glove.jsz","r") do fh
    deserialize(fh)    
end

#data = open("atis_data2.jsz","r") do fh
#    deserialize(fh)    
#end

#const zeroed_words = data["zeroed_words"]
@everywhere zeroed_words = ASCIIString[]

In [44]:
short_cases = Bool[length(ws) <=10 for ws in unordered_output]

eval_cases = [1:100]
true_ordered_sents = test_set[short_cases][eval_cases]
ordered_sents = pmap(unordered_output[short_cases][eval_cases]) do unordered_words
    order(unordered_words, zeroed_words)
end


100-element Array{Any,1}:
 ASCIIString(i, want, to, go, from, denver, to, oakland)                        
 ASCIIString(what, is, the, cheapest, flight, from, denver, to, oakland)        
 ASCIIString(one, way, please)                                                  
 ASCIIString(i, want, to, leave, after, five, p, m)                             
 ASCIIString(sure, i, want, to, go, from, philadelphia, to, dallas)             
 ASCIIString(on, which, flights, will, lunch, be, served)                       
 ASCIIString(which, flights, serve, lunch)                                      
 ASCIIString(thank, you)                                                        
 ASCIIString(i, live, in, denver)                                               
 ASCIIString(what, is, the, least, expensive, flight, from, atlanta, to, denver)
 ASCIIString(is, there, a, round, trip, flight, that, stops, in, pittsburgh)    
 ASCIIString(round, trip, please)                                               
 A

In [10]:
perfect_matches = Bool[]  
for ii in 1:length(ordered_sents)
    ordered_words = ordered_sents[ii]
    actual_words = true_ordered_sents[ii]
    
    match = ordered_words == actual_words
    push!(perfect_matches, match)
    println("$ii - $match")
end
mean(perfect_matches)

1 - true


0.4

In [11]:
@pyimport nltk
@pyimport nltk.translate.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate #Perfect Match
        1.0
    else
        weights = [1,1,1,1]/4
        nltk_bleu.bleu(Any[reference],candidate, weights)
    end
end

2 - false
3 - false
4 - true
5 - true
6 - false
7 - false
8 - false
9 - false
10 - false
11 - false
12 - true
13 - true
14 - true
15 - true
16 - false
17 - false
18 - false
19 - true
20 - false


bleu_score (generic function with 1 method)

In [13]:
map(bleu_score, ordered_sents[1:40],true_ordered_sents[1:40]) |> mean


LoadError: LoadError: BoundsError: attempt to access 20-element Array{Any,1}:
 ASCIIString(i, want, to, go, from, denver, to, oakland)                                                                                                                                                                                                                                                                                                                                                                                               
 RemoteException(3,CapturedException(BoundsError(Tuple{FunctionalCollections.PersistentList{ASCIIString},Float64}[],(1,)),Any[(:getindex,symbol("array.jl"),272,symbol(""),-1,1),(:order,symbol("In[6]"),13,symbol(""),-1,1),(:anonymous,symbol("In[9]"),5,symbol(""),-1,1),(:anonymous,symbol("multi.jl"),892,symbol(""),-1,1),(:run_work_thunk,symbol("multi.jl"),645,symbol(""),-1,1),(:anonymous,symbol("multi.jl"),892,symbol("task.jl"),59,1)])) 
 RemoteException(4,CapturedException(Bound

In [ ]:
bleu_score(true_ordered_sents[3], true_ordered_sents[3])

In [ ]:
true_ordered_sents

In [ ]:
ordered_sents

In [ ]:
ordered_sents[50]

In [ ]:
unordered_output[short_cases][eval_cases][50]

In [ ]:
true_ordered_sents[50]

In [ ]:
#x= ["A", "B", "C", "D"]
#y= UTF8String["A", "B", "C", "D"]
x = true_ordered_sents[10]
y=ordered_sents[10] |> collect
pycall(nltk_bleu.bleu, PyAny, Any[x], y, Any[0.25, 0.25, 0.25, 0.25])

In [ ]:
Any[true_ordered_sents[1]]


In [ ]:
nltk_bleu._modified_precision(Any[reference1, reference2, reference3],candidate1, 1)

In [ ]:
@pyimport pdb

In [ ]:
pdb.runcall(nltk_bleu._modified_precision, [reference1, reference2, reference3],candidate1, 1)

In [ ]:
# I think I have to reimplement BLEU in julia as for some reason it does not play nice with PyCall
# Can basically port http://www.nltk.org/_modules/nltk/align/bleu_score.html#bleu

In [ ]:
pycall(nltk_bleu.bleu, Int, candidate1, [reference1], weights)

In [ ]:
@pyimport nltk.util as nltk_util
nltk_util.ngrams(candidate1,2) |> py_collections.Counter

In [ ]:
@pyimport collections as py_collections

In [ ]:
@pyimport nltk.util as nltk_util
ngs = nltk_util.ngrams(candidate1,2)
pycall(py_collections.Counter, PyObject, ngs)

In [ ]:
import nltk